### See solution_array_of_change below, I tried that and got OK performance, but not perfect... it's actually a recursive solution as well

### Idea: Recursive solution
The substring to be removed either starts from left edge, right edge or is a chunk in the middle.

Solution = min(three solutions)

In [219]:
def idx_chop_left(S):
    ''' find index of second character in S 
        Example: S="abc", "a" appears once, return 1 ("b" != "a")
        Example: S="aabc", "a" appears twice, return 2 
    '''
    char = S[0]
    idx_next=1
    while S[idx_next] == char:
        idx_next += 1
    return idx_next
    

In [220]:
def solution_left(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
        Substring starts from beginning of S 
    '''
    if K == 0: return len(S)
    if len(set(S)) == K: return 0 # SUCCESS: no removal needed
    idx_new_left = idx_chop_left(S)
    return idx_new_left + solution_left(S[idx_new_left:], K)

In [221]:
def test_solution_left():
    test_set = [('aabc', 2),
                ('aa', 1),
                ('asdf', 0), 
                ('foobar', 3),
               ('asdfaaamoo', 4)]
    for test in test_set:
        print("S = {}, K = {}, solution_left = {}".format(test[0], test[1], solution_left(test[0], test[1])))

test_solution_left()

S = aabc, K = 2, solution_left = 2
S = aa, K = 1, solution_left = 0
S = asdf, K = 0, solution_left = 4
S = foobar, K = 3, solution_left = 3
S = asdfaaamoo, K = 4, solution_left = 3


In [222]:
def idx_chop_right(S):
    ''' Find index of first new character in S, counting from the end
        Example: S="abc", "c" appears once, index = 2
        Example: S="abccc", "c", appears three times, index = 2
    '''
    char=S[-1]
    idx_next = len(S)-2
    while S[idx_next] == char:
        idx_next -= 1
    return idx_next+1
    

In [223]:
def solution_right(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
        Substring starts from end of S (walking from right to left) '''
    if K == 0: return len(S)
    if len(set(S)) == K: return 0 # SUCCESS: no removal needed
    idx_right = idx_chop_right(S)
    #print("S={},idx_right={},S[0:idx_right]={}".format(S,idx_right,S[0:idx_right]))
    return len(S) - idx_right + solution_right(S[0:idx_right], K)

In [224]:
def test_solution_right():
    test_set = [('aabc', 2),
                ('aa', 1),
                ('asdf', 0), 
                ('foobar', 3),
               ('asdfaaamoo', 4)]
    for test in test_set:
        print("S = {}, K = {}, solution_right = {}".format(test[0], test[1], solution_right(test[0], test[1])))

test_solution_right()

S = aabc, K = 2, solution_right = 1
S = aa, K = 1, solution_right = 0
S = asdf, K = 0, solution_right = 4
S = foobar, K = 3, solution_right = 2
S = asdfaaamoo, K = 4, solution_right = 3


In [225]:
def solution_middle_test(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
        Substring does not include left and right edges of S '''
    if K == 0: return len(S)
    if len(set(S)) == K: return 0 # SUCCESS: no removal needed
    #if len(S) < 3:
    #    return 3 # use a too large value to signal that no middle solution exists
    
    #(char_start, char_end) = (S[0], S[-1])
    #char_edges = set((char_start, char_end))
    char_edges = set((S[0], S[-1]))
    
    (idx_left, idx_right) = chop_edges(S)
    return solution(S[idx_left: idx_right], K - len(char_edges))

In [226]:
def solution_recursive(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
    '''
    if len(set(S)) < K: return -1 # S doesn't contain enough different characters. Example S='acc', K=3
    if K==0: return len(S)        # remove full string 
    return min(solution_left(S, K), solution_right(S, K), solution_middle(S, K))

In [227]:
def test_solution_recursive():
    test_set = [('aabc', 2),
                ('aa', 1),
                ('asdf', 0), 
                ('foobar', 3),
               ('oooooobbbbbbasdfaam', 4),
               ('baaaaaa', 1)]
    for test in test_set:
        print("S = {}, K = {}, solution = {}".format(test[0], test[1], solution_recursive(test[0], test[1])))
test_solution_recursive()

sol_middle: S=aabc
remainder=aac
chop_maximum_from_right:: S = b, K=2, charset_remainder={'c', 'a'}
about to return remaining (post chopped-off) length of string = 1
do_middle = True


IndexError: string index out of range

### Idea2: simpler linear solution
Scan the string with a sliding window method, noting down characters already used.

In [ ]:
def solution_sliding_window(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
    '''
    char_S = set(S)
    if len(char_S) < K: return -1 # S doesn't contain enough different characters. Example S='acc', K=3
    if K==0: return len(S)        # remove full string
    
    nchars_remove = len(char_S) - K
    #len_substr_max = len(S)
    len_substr_min = nchars_remove # if we have to strip S of say 3 characters, the substring to remove must be at least this long
    idx_left = 0; idx_right = len_substr_min;
    


### Idea3: array of change
Scan the string and mark indices at which there's a change in the number of characters

In [229]:
def solution_remove_from_right(S, indices_change, K):
    ''' indices_change markes indices where new characters appear in S, starting from the left.
        Return shortest substring yielding K different characters in remaining string, starting from end (right edge)
        
        Example: S="aabccab" will have indices_change [0, 2, 3], with say K=2 we return 'ccab'
    '''
    #return len(S)-indices_change[K]
    return S[indices_change[K]:]
    

In [230]:
def solution_remove_from_left(S, indices_change, K):
    ''' given array marking indices of change, return shortest substring to be removed starting from beginning,
        leaving S with K different characters
        
        Example: S="rytabbjj" will have indices_change [7, 5, 3, 2, 1, 0], with say K=2 we return 'ryta' 
    '''    
    #return indices_change[K]+1
    return S[0:indices_change[K]+1]

In [291]:
def chop_maximum_from_left(S, K, charset_remainder):
    ''' chop S from left end, until len(charset_remainder) exceeds K.
        Return remaining string, the one to be removed from S
    '''
    ## print-out
    #print("chop_maximum_from_left:: S = {}, K={}, charset_remainder={}".format(S, K, charset_remainder))
    ## 
    
    for idx_left in range(len(S)):
        if len(charset_remainder) > K:
            break
        charset_remainder.add(S[idx_left])
    idx_left -= 1
    #while len(charset_remainder) <= K:
    #    charset_remainder.add(S[idx_left])
    #    idx_left += 1
    #idx_left -= 1 # take one step back to revert exceeding
        #print("before next while: charset_remainder={}, idx_left={}".format(charset_remainder,idx_left))

    #print("about to return remaining string = {}".format(S[idx_left:]))
    return S[idx_left:] 
### length of string
        

In [314]:
def chop_maximum_from_right(S, K, charset_remainder):
    ''' chop S from end, until len(charset_remainder) exceeds K.
        Return remaining string, to be removed from S
    '''
    ### print-out
    #print("chop_maximum_from_right:: S = {}, K={}, charset_remainder={}".format(S, K, charset_remainder))
    ###
    for idx_right in range(len(S)-1, -1, -1):
        if len(charset_remainder) > K:
            break
        charset_remainder.add(S[idx_right]) #= len(S)-1
    #while len(charset_remainder) <= K:
    #    charset_remainder.add(S[idx_right])
    #    idx_right -= 1
    idx_right += 1 # revert exceeding 
    #str_len=idx_right+1
    #print("idx_right = {}".format(idx_right))
    #print("remaining string: {}".format(S[:idx_right+1]))
    #print("about to return remaining string = {}".format(S[:idx_right+1]))
    return S[:idx_right+1]
        

In [315]:
def solution_middle(S, K, charset=set()): #, len_remainder=0): #, char_S):
    ''' Find shortest substring to remove from S, not including its edges,
        leaving remainder with K different characters.
        Remainder from previous iterations already contains characters in charset
    '''
    #print("inside solution_middle, S={}, K={}".format(S, K))
    if K==0: return len(S)
    #char_S=set(S)
    #if len(char_S) < K: return len(S)+1 # signify no solution with too large value
    #elif len(char_S) == K:
    #    print("HERE")
    #    return 0 # SUCCESS no need to remove anything
    
    # remove edges
    # START (left edge):
    char_start = S[0]
    char_end = S[-1]
    # check that chopping off edges won't exceed K
    if len(charset.union((char_start, char_end))) > K:
        #print("HALLON")
        return S
    
    idx_left = 1
    while S[idx_left] == char_start:
        idx_left += 1
    # END (right edge):

    idx_right = len(S)-2
    while S[idx_right] == char_end:
        idx_right -= 1
    idx_right += 1
    
    
    if idx_left == idx_right:
        print("idx_left == idx_right!!!")
        return len(S)+1 # signify no solution with too large value
    
    #print("idx_left = {}, idx_right = {}".format(idx_left, idx_right))
    ### print-outs
    #print(">>>> sol_middle called with S={}, charset={}".format(S, charset))
    remainder = S[0:idx_left] + S[idx_right:]
    #print("Inside sol_middle() :: removed edges = {}, continuing with {}".format(remainder, S[idx_left:idx_right]))
    ###
    
    # update charset
    charset = charset.union(set(remainder))
    # two solutions: chop from left or right until done
    #print("** CHOP FROM LEFT::")
    sol_rightedge_fixed =  chop_maximum_from_left(S[idx_left:idx_right], K, set(charset)) 
    #print("** CHOP FROM RIGHT::")
    sol_leftedge_fixed = chop_maximum_from_right(S[idx_left:idx_right], K, set(charset)) 
    # check if we are to continue to look for middle solution: update charset
    
    do_middle = len(charset) <= K # can't chop off edges without exceeding K, recursion stops
    ### print-outs
    #print("do_middle = {}".format(do_middle))
    ###
    
    ####
    if do_middle:
        sol_middle = solution_middle(S[idx_left:idx_right], K, charset) #, len(remainder))
        #print("sol_rightedge_fixed={},\n sol_leftedge_fixed={},\n sol_middle={}\n".format(sol_rightedge_fixed, sol_leftedge_fixed, sol_middle))
        lengths = [len(sol_leftedge_fixed), len(sol_middle), len(sol_rightedge_fixed)]
        idx_min = lengths.index(min(lengths))
        if idx_min==0: return sol_leftedge_fixed
        elif idx_min==1: return sol_middle
        else: return sol_rightedge_fixed
        
        #return min(sol_rightedge_fixed,
                              #sol_leftedge_fixed,
                              #sol_middle)
    else:
        len_right =  len(sol_rightedge_fixed)
        len_left  =  len(sol_leftedge_fixed)
        return sol_leftedge_fixed if len_left<=len_right else sol_rightedge_fixed
            

    
    
    
    
    #print("sol_rightedge_fixed = {}".format(sol_rightedge_fixed))
    #return sol_rightedge_fixed
    
    

In [316]:
def solution_array_of_change(S, K):
    ''' Find shortest substring to remove, leaving S with K different characters.
    '''
    if K==0: return len(S)        # remove full string
    char_S = set(S)
    if len(char_S) < K: return -1 # S doesn't contain enough different characters. Example S='acc', K=3
    elif len(char_S) == K: return 0 # SUCCESS no need to remove anything

    # scan array from left
    indices_change_left = [0] # first char is new char
    characters = set(S[0])
    for idx in range(1, len(S)):
        if S[idx] not in characters:
            characters.add(S[idx])
            indices_change_left.append(idx)
    if not characters == char_S:
        print("PROBLEM in scanning array")
            
    # scan array from right
    indices_change_right = [len(S)-1] # first char is new char
    characters = set(S[-1])
    for idx in range(len(S)-1, -1, -1):
        if S[idx] not in characters:
            characters.add(S[idx])
            indices_change_right.append(idx)
            
    if not characters == char_S:
        print("PROBLEM in scanning array")
        
    
    (sol_left, sol_right, sol_middle) = (solution_remove_from_right(S, indices_change_left, K),
                                      solution_remove_from_left(S, indices_change_right, K) ,
                                      solution_middle(S, K))
  
    print("HERE")
    print("sol_left = {},\nsol_right = {},\nsol_middle={}".format(sol_left,sol_right,sol_middle))


    
    
    

In [317]:
def tests_solution_full():
    test_set = [('oooooobbbbbbasdfaam', 4),
              ('foobar', 3),
               ('aabc', 2),
               ('aa', 1),
               ('asdf', 0), 
               
              
              ('baaaaaa', 1),
              ('rytabbjj', 3)
               ]
    for test in test_set:
        print("S = {}, K = {}".format(test[0], test[1]))
        print("solution: \n")
        solution_array_of_change(test[0], test[1])
        print("")
        
tests_solution_full()

S = oooooobbbbbbasdfaam, K = 4
solution: 

HERE
sol_left = dfaam,
sol_right = oooooobbbbbbas,
sol_middle=sdf

S = foobar, K = 3
solution: 

HERE
sol_left = ar,
sol_right = foo,
sol_middle=ba

S = aabc, K = 2
solution: 

HERE
sol_left = c,
sol_right = aa,
sol_middle=b

S = aa, K = 1
solution: 


S = asdf, K = 0
solution: 


S = baaaaaa, K = 1
solution: 

HERE
sol_left = aaaaaa,
sol_right = b,
sol_middle=baaaaaa

S = rytabbjj, K = 3
solution: 

HERE
sol_left = abbjj,
sol_right = ryt,
sol_middle=yta



In [ ]:
def tests_solution_middle():
    test_set = [('aabc', 2),
                ('aa', 1),
                ('asdf', 0), 
                ('foobar', 3),
               ('oooooobbbbbbasdfaam', 4),
               ('baaaaaa', 1),
               ('rytabbjj', 3)]
    for test in test_set:
        print("S = {}, K = {}".format(test[0], test[1]))
        print("solution_middle = {}".format( solution_middle(test[0], test[1])))
        print("")
        
tests_solution_middle()

In [52]:
def test_solution_left_array():
    test_set = [('aabc', 2),
                ('aa', 1),
                ('asdf', 0), 
                ('foobar', 3),
               ('oooooobbbbbbasdfaam', 4),
               ('baaaaaa', 1)]
    for test in test_set:
        print("S = {}, K = {}, solution = {}".format(test[0], test[1], solution_array_of_change(test[0], test[1])))
test_solution_left_array()

S = aabc, K = 2, solution = 1
S = aa, K = 1, solution = 0
S = asdf, K = 0, solution = 4
S = foobar, K = 3, solution = 2
S = oooooobbbbbbasdfaam, K = 4, solution = 5
S = baaaaaa, K = 1, solution = 1
